# Training a PyTorch MNIST model with MlFlow and Optuna

## Imports

In [34]:
from __future__ import print_function
import os
import time

import optuna
import mlflow
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from mlflow import pytorch
from urllib.parse import urlparse

## Network 

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

## Train and validation functions

In [36]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    train_set_size = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            # Log batch loss using mlflow
            mlflow.log_metric("train_batch_loss", loss.item())
            batch_size = len(data)
            print(f"Train Epoch: {epoch} [{batch_idx * batch_size}/{train_set_size} "
                  f"({100. * batch_idx / train_set_size:.0f}%)]\tLoss: {loss.item():.6f}")


def test(model, device, test_loader):
    model.eval()
    test_set_size = len(test_loader.dataset)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_set_size
    mlflow.log_metric("test_loss", test_loss)

    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{test_set_size} "
          f"({100. * correct / test_set_size:.0f}%)")

## Main

In [37]:
def get_artifact_path(active_run):
    parsed_uri = urlparse(active_run.info.artifact_uri)
    artifact_path = os.path.abspath(os.path.join(parsed_uri.netloc, parsed_uri.path))
    return artifact_path

In [38]:
def objective(trial, experiment, params=None):
    # Start mlflow run
    with mlflow.start_run(experiment_id=experiment.experiment_id):
        active_run = mlflow.active_run()
        artifact_path = get_artifact_path(active_run)
        print(f"Artifact path for this run: {artifact_path}")
        
        # Use mlflow to log params
        mlflow.log_params(params)

        # Use CUDA if GPU is available
        use_cuda = params["use_cuda"] and torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        # Log mlflow device parameter
        mlflow.log_param("device", device)

        torch.manual_seed(params["seed"])

        # Load the MNIST train and test dataset and save it to ./data
        mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.1307,), (0.3081,))
                                       ]))
        train_loader = torch.utils.data.DataLoader(mnist_train,
                                                   batch_size=params["batch_size"],
                                                   shuffle=True)
        mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.1307,), (0.3081,))
                                       ]))
        test_loader = torch.utils.data.DataLoader(mnist_test,
                                                  batch_size=params["validation_batch_size"],
                                                  shuffle=True)

        model = Net().to(device)
        optimizer = optim.Adam(model.parameters(), lr=params["lr"])
        scheduler = StepLR(optimizer, step_size=1, gamma=params["gamma"])

        for epoch in range(0, params["epochs"]):
            train(params, model, device, train_loader, optimizer, epoch)
            test(model, device, test_loader)
            scheduler.step()
    
        if params["save_model"]:
            pytorch.save_model(model, f"{artifact_path}/mnist_model")

In [39]:
def main(params=None):
    # create mlflow experiment if it doesn't exist already
    experiment_name = params["experiment_name"]
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        mlflow.create_experiment(experiment_name)
        experiment = mlflow.get_experiment_by_name(experiment_name)
    mlflow.set_experiment(experiment_name)

    # Create the optuna study whih has the same name as the experiment
    study = optuna.create_study(study_name=experiment_name, direction="minimize")
    study.optimize(lambda trial: objective(trial, experiment, params), n_trials=5)

    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [40]:
PARAMS = {
    "experiment_name": "pytorch-optuna-mlflow",
    "batch_size": 128,
    "validation_batch_size": 1000,
    "epochs": 3,
    "lr": 1.0,
    "gamma": 0.7,
    "use_cuda": False,
    "seed": 42,
    "log_interval": 10,
    "save_model": True
}
main(params=PARAMS)

Artifact path for this run: /home/steffi/dev/blog/pytorch-mlflow-optuna/mlruns/0/d951887ff4c0407db6d06574d378269e/artifacts
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.330135
Train Epoch: 0 [1280/60000 (0%)]	Loss: 3.657842
Train Epoch: 0 [2560/60000 (0%)]	Loss: 2.596306
Train Epoch: 0 [3840/60000 (0%)]	Loss: 2.328610
Train Epoch: 0 [5120/60000 (0%)]	Loss: 2.325323
Train Epoch: 0 [6400/60000 (0%)]	Loss: 2.343570
Train Epoch: 0 [7680/60000 (0%)]	Loss: 2.347873
Train Epoch: 0 [8960/60000 (0%)]	Loss: 2.345505
Train Epoch: 0 [10240/60000 (0%)]	Loss: 2.312938
Train Epoch: 0 [11520/60000 (0%)]	Loss: 2.287150
Train Epoch: 0 [12800/60000 (0%)]	Loss: 2.350554
Train Epoch: 0 [14080/60000 (0%)]	Loss: 2.364367
Train Epoch: 0 [15360/60000 (0%)]	Loss: 2.350597
Train Epoch: 0 [16640/60000 (0%)]	Loss: 2.422240
Train Epoch: 0 [17920/60000 (0%)]	Loss: 2.316475
Train Epoch: 0 [19200/60000 (0%)]	Loss: 2.314355
Train Epoch: 0 [20480/60000 (0%)]	Loss: 2.339597
Train Epoch: 0 [21760/60000 (0%)]	Loss: 2.380643
Train

KeyboardInterrupt: 